In [32]:
#Import der relevanten Module

import ifcopenshell
import ifcopenshell.util
import ifcopenshell.util.element

import pandas as pd

#Helpers - Hilfs-Datenstrukturen erstellen

import Helpers as hlp
import Export_Data as ed


#Hilfsfunktion zum filtern

def filterConfig(_cat_name):
    try:
        serach_res = [x for x in conf_lst if x.category == _cat_name][0]
    except:
        serach_res = None
        
    return serach_res

#Globale Variablen - Listen

conf_lst = []
export_lst = []

#Alle Grundlagen einlesen > IFC + Excel-Elementplan

print("Start: Starte Grunddatein einlesen...")

source_path = "./ARC_Modell_NEST_230328.ifc"
ifc_file = ifcopenshell.open(source_path)

xls_file = ("./Elementplan.xlsx")

s_data_category = pd.read_excel(xls_file, "Objektkatalog")
s_data_attributes = pd.read_excel(xls_file, "Attributsliste")


#Daten aus dem Elementplan auslesen und strukturiert speichern

for index, row in s_data_category.iterrows():  

    attr_df = s_data_category.iloc[index, 2]

    if attr_df:

        attr_in_group = s_data_attributes[s_data_attributes["Gruppe"] == attr_df]
        
        print("Klasse: " + row["IfcClass"])
        print(attr_in_group)
        print("Next")

        prop_lst = []

        for _i, _r in attr_in_group.iterrows():

            p_h = hlp.Prop_Holder(_r["Pset"], _r["Property"])
            prop_lst.append(p_h)


        _target_obj = filterConfig(row["IfcClass"])

        if _target_obj != None:
            _target_obj.prop_list.append(prop_lst)
        else:
            d_h = hlp.Data_Holder(row["IfcClass"], prop_lst)
            conf_lst.append(d_h)


#IFC Datein Auswerten und Daten strukturiert speichern

for _c in conf_lst:
    
    elements = ifc_file.by_type(_c.category)
    
    tmp_exp_data_from_current_config = []
    
    for _e in elements:
        #print(_e.Name)
        
        tmp_exp_data = []
        try:
            tmp_exp_data.append(_e.get_info().get("type"))
        except:
            continue
        
        act_prop = ifcopenshell.util.element.get_psets(_e, psets_only=True)
        
        for _p in _c.prop_list:
            #print(_p.prop)
            try:
                act_val = act_prop.get(_p.pset).get(_p.prop)
                #print(act_val)
                tmp_exp_data.append(act_val)
            except:
                act_val = ""
                tmp_exp_data.append(act_val)
                
        tmp_exp_data_from_current_config.append(tmp_exp_data)
        #print(tmp_exp_data_from_current_config)
                
    
    exp_holder = ed.Exp_Holder(_c.category, tmp_exp_data_from_current_config)    
    export_lst.append(exp_holder)

#Export der Daten als Excel

exp_path = "./Export_Daten_IFC4.xlsx"
pd.DataFrame([]).to_excel(exp_path)

#Export Extracted Data to Excel

if len(export_lst) > 0:

    for exp_set in export_lst:
    
        col_lst = []
        col_lst.append("Kategorie")
        
        _target_obj = filterConfig(exp_set.branch)
        
        if _target_obj != None:
            
            for _p_l in _target_obj.prop_list:
                _ps = _p_l.pset
                _pr = _p_l.prop

                #_title = _ps + ":" + _pr
                _title = "{}:{}".format(_ps, _pr)
                
                col_lst.append(_title)
                #print(_title)


            df = pd.DataFrame(exp_set.data, columns = col_lst)
            #print(exp_set.data)
            #df = pd.DataFrame(exp_set.data)
            
            with pd.ExcelWriter(exp_path, engine='openpyxl', if_sheet_exists='replace', mode='a') as writer:  
                df.to_excel(writer, sheet_name= exp_set.branch, index=False)
            

    print("Final Step: Finished Export Excel...")


Start: Starte Grunddatein einlesen...
Klasse: IfcSpace
   Pset                  Property          Gruppe
0  NEST  Flächenkategorie SIA 416  Basisattribute
1  NEST                  Raumname  Basisattribute
2  NEST                Raumnummer  Basisattribute
3  NEST                  Geschoss  Basisattribute
Next
Klasse: IfcElement
              Pset    Property         Gruppe
4             NEST      eBKP-H  BasisBauteile
5  Pset_SlabCommon  IsExternal  BasisBauteile
Next
Final Step: Finished Export Excel...
